In [4]:
import numpy as np
import wandb
import torch
from dc_ldm.util import instantiate_from_config

import torch.nn as nn
import os


#Modifications
from dc_ldm.models.EEGNet_Embedding_version import EEGNet_Embedding
import yaml
# from dc_ldm.ldm_for_eeg import 

def read_model_config(config_path: str):
    with open(config_path) as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
        config = config["parameters"]["model"]["parameters"]
        model_config = {key: config[key]["value"] for key in config}
    return model_config

def updated_forward(self, x):
    x = self.partial_forward(x) # [1 16 1 15] #bs x F2 x 1 x 15 (128*15)
    x = x.flatten(start_dim=1) # bs x F2*15
    x = self.embedding(x) # bs x 512
    return x
EEGNet_config_path = "C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\Generation\\reconstruction\\pretrains\\EEG\\P001_model_config.yaml"
# EEGNet_ckpt_path = "C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\Generation\\reconstruction\\pretrains\\EEG\\best-model-epoch=03-val_acc=0.26.ckpt"
EEGNet_ckpt_path = "C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\Generation\\pytorch\\results\\wandb-logs\\EEGNet_Embedding_512_P001_final\\lnh3x1m6\\checkpoints\\best-model-epoch=14-val_acc=0.26.ckpt"
print(f"the path from eeg is {EEGNet_ckpt_path}")
model_config = read_model_config(config_path = EEGNet_config_path) #"C:/Users/s_gue/Desktop/master_project/sven-thesis/pytorch/EEG_encoder_setup/P001_model_config.yaml")
model = EEGNet_Embedding(**model_config) #This is the EEGNet model mapping from EEG input to embedding
checkpoint = torch.load(EEGNet_ckpt_path)
model.load_state_dict(checkpoint['state_dict'])
model.forward = updated_forward.__get__(model)

print(model)


the path from eeg is C:\Users\mituser\Desktop\3DReconstruction\imagery2024\Generation\pytorch\results\wandb-logs\EEGNet_Embedding_512_P001_final\d5ia8swl\checkpoints\best-model-epoch=00-val_acc=0.18.ckpt
this is the size <built-in method size of Tensor object at 0x00000241DBBC99D0> 
EEGNet_Embedding(
  (loss): NLLLoss()
  (acc): MulticlassAccuracy()
  (ensuredims): Ensure4d()
  (dimshuffle): Expression(expression=_transpose_to_b_1_c_0) 
  (conv_temporal): Conv2d(1, 64, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
  (bnorm_temporal): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (conv_spatial): Conv2dWithConstraint(64, 128, kernel_size=(64, 1), stride=(1, 1), groups=64, bias=False)
  (bnorm_1): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (elu_1): Expression(expression=elu) 
  (pool_1): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
  (drop_1): Dropout(p=0

C:\Users\mituser\AppData\Local\Temp\ipykernel_22084\339497619.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(EEGNet_ckpt_path)


In [10]:
import torch
ckp_path = "C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\Generation\\reconstruction\\pretrains\\ldm\\label2img\\model.ckpt"
checkpoint = torch.load(ckp_path)
# print(checkpoint.keys())
# print(checkpoint['state_dict'].keys())
model_keys = [key for key in checkpoint['state_dict'].keys() if key.startswith('model')]
# print(model_keys)
# for key in model_keys:
#     print(f"{key}: {checkpoint['state_dict'][key]}")

for key in model_keys:
    print(f"{key}: {checkpoint['state_dict'][key].shape}")

C:\Users\mituser\AppData\Local\Temp\ipykernel_20396\123354552.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckp_path)


model.diffusion_model.time_embed.0.weight: torch.Size([768, 192])
model.diffusion_model.time_embed.0.bias: torch.Size([768])
model.diffusion_model.time_embed.2.weight: torch.Size([768, 768])
model.diffusion_model.time_embed.2.bias: torch.Size([768])
model.diffusion_model.input_blocks.0.0.weight: torch.Size([192, 3, 3, 3])
model.diffusion_model.input_blocks.0.0.bias: torch.Size([192])
model.diffusion_model.input_blocks.1.0.in_layers.0.weight: torch.Size([192])
model.diffusion_model.input_blocks.1.0.in_layers.0.bias: torch.Size([192])
model.diffusion_model.input_blocks.1.0.in_layers.2.weight: torch.Size([192, 192, 3, 3])
model.diffusion_model.input_blocks.1.0.in_layers.2.bias: torch.Size([192])
model.diffusion_model.input_blocks.1.0.emb_layers.1.weight: torch.Size([192, 768])
model.diffusion_model.input_blocks.1.0.emb_layers.1.bias: torch.Size([192])
model.diffusion_model.input_blocks.1.0.out_layers.0.weight: torch.Size([192])
model.diffusion_model.input_blocks.1.0.out_layers.0.bias: tor

In [16]:
import pandas as pd
import numpy as np
import os

df = pd.read_excel("C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\Image3DObjects\\3Dimages_Windows.xlsx", engine='openpyxl')
    
    # Assuming the column `images` contains relative paths or full paths
base_path = "C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\Image3DObjects"

# Correct path concatenation - handle if the path is relative or absolute
image_paths = np.array([
    os.path.join(base_path, img) if not os.path.isabs(img) else img 
    for img in df['images'].values
])

# Ensure image paths are valid
for img_path in image_paths:
    print(f"Warning: File does not exist - {img_path}")


In [10]:
import numpy as np
eeg_val_path = np.load("C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\DATA\\S02\\val\\data.npy")
eeg_val_path.shape


(38, 64, 4096)

In [6]:
import numpy as np
a = np.load("C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\DATA\\S01\\seclevel\\train\\data.npy")
b = np.load("C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\DATA\\S01\\seclevel\\val\\data.npy")
c = np.load("C:\\Users\\mituser\\Desktop\\3DReconstruction\\imagery2024\\DATA\\S01\\seclevel\\train\\labels.npy")
print(a.shape)
print(b.shape)
c

(3369, 64, 512)
(187, 64, 512)


array([239, 132, 241, ...,  12, 115, 510], dtype=int64)